# Setup and Installations

In [ ]:
import os
import subprocess

# Define the setup script path
setup_script_path = 'setup_notebook.sh'

# Run the setup script
subprocess.call(['bash', setup_script_path])

# Print environment setup confirmation
print("Environment setup complete.")


In [ ]:
import stable_baselines3
import matplotlib
import traci
import sumo_rl
import imageio
import numpy as np
import tensorboard
import os
import sys
import gymnasium as gym
from stable_baselines3.dqn.dqn import DQN
import traci
from sumo_rl import SumoEnvironment
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
from stable_baselines3 import DQN
import imageio.v2 as imageio
from IPython.display import Video
import pandas as pd

In [105]:
print("SUMO_HOME:", os.environ.get("SUMO_HOME"))

SUMO_HOME: /opt/homebrew/opt/sumo/share/sumo


In [107]:

# Get the path to the installed sumo-rl package
sumo_rl_path = os.path.dirname(sumo_rl.__file__)
print("Path to sumo-rl package:", sumo_rl_path)

# Optional: List the contents of the package directory
for root, dirs, files in os.walk(sumo_rl_path):
    for file in files:
        print(os.path.join(root, file))


Path to sumo-rl package: /Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/.DS_Store
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/__init__.py
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/util/__init__.py
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/util/gen_route.py
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/util/__pycache__/gen_route.cpython-311.pyc
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/util/__pycache__/__init__.cpython-311.pyc
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/agents/ql_agent.py
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/agents/__init__.py
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/agents/__pycache__/ql_agent.cpython-311.pyc
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/agents/__pycache__/__init__.cpython-311.pyc
/Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/__pycache__/__init__.cp

# Environment Setup

In [108]:
if "SUMO_HOME" in os.environ:
    tools = os.path.join(os.environ["SUMO_HOME"], "tools")
    sys.path.append(tools)
else:
    sys.exit("Please declare the environment variable 'SUMO_HOME'")



## Setup Timestamp for Training

In [109]:

jerusalem_tz = pytz.timezone('Asia/Jerusalem')

jerusalem_time = datetime.now(jerusalem_tz)

# Format the timestamp
timestamp = jerusalem_time.strftime("%Y%m%d_%H%M%S")

## SUMO Package Path and Configuration File


In [110]:
package_path = os.path.dirname(sumo_rl.__file__)
sumo_cfg_path = os.path.join(package_path, "nets/single-intersection/single-intersection.sumocfg")


## Environment preparation for the Agent training

In [111]:
env_dqn_agent = SumoEnvironment(
    net_file=f"{package_path}/nets/single-intersection/single-intersection.net.xml",
    route_file=f"{package_path}/nets/single-intersection/single-intersection.rou.xml",
    out_csv_name=f"Outputs/single-intersection/dqn_{timestamp}.csv",
    single_agent=True,
    use_gui=True,
    num_seconds=3600,
)
print("SUMO environment for agent training was created")

 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                      
SUMO environment created


## Environment preparation for the Naive solution

In [ ]:
env_fixed_policy = SumoEnvironment(
        net_file=f"{package_path}/nets/single-intersection/single-intersection.net.xml",
        route_file=f"{package_path}/nets/single-intersection/single-intersection.rou.xml",
        out_csv_name = f"Outputs/single-intersection/dqn_{timestamp}.csv",
        single_agent=True,
        use_gui=True,
        num_seconds=3600,
        min_green=42,
        max_green=42,
        yellow_time=2
    )

print("SUMO environment for naive solution was created")

# Agent Setup

In [112]:
rl_model = DQN(
    env=env_dqn_agent,
    policy="MlpPolicy",
    learning_rate=0.001,
    learning_starts=0,
    train_freq=1,
    target_update_interval=1,
    exploration_initial_eps=0.05,
    exploration_final_eps=0.01,
    verbose=2,
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# Naive Solution Setup

In [ ]:
naive_model = DQN(
    env=env_fixed_policy,
    policy="MlpPolicy",
    learning_rate=0.00000000000001,
    learning_starts=0,
    train_freq=1,
    target_update_interval=1,
    exploration_initial_eps=0.05,
    exploration_final_eps=0.01,
    verbose=2,
)

sumo_binary = '/opt/homebrew/Cellar/sumo/1.20.0/bin/sumo-gui'

# Training the agent

## Run the following code to get the path to the SUMO remote server


In [113]:
# Use the package path in your command
if package_path:
    cmd = f"sumo-gui -c {sumo_cfg_path} --remote-port 65533"
    print("Run the following command in the terminal to start the SUMO server:")
    print(cmd)
else:
    print("sumo-rl package not found. Make sure it is installed.")

Run the following command in the terminal to start the SUMO server:
sumo-gui -c /Users/md/anaconda3/lib/python3.11/site-packages/sumo_rl/nets/single-intersection/single-intersection.sumocfg --remote-port 65533


In [86]:
if __name__ == "__main__":
    print("Connecting to SUMO server...")
    traci.connect(port=65533)
    print("Connected to SUMO server")
    
    num_of_episodes = 80
    print("Starting training")
    rl_model.learn(total_timesteps=(720*num_of_episodes))
    print("Training completed")

    # Save the model
    rl_model.save(f'savedAgent/single-intersection/dqnEpNum{num_of_episodes}')
    print("Model saved")

    # Close the TraCI connection
    traci.close()
    print("TraCI connection closed")


Connecting to SUMO server...
Connected to SUMO server
Starting training
 Retrying in 1 seconds
Training completed
Model saved
TraCI connection closed


# Agent Prediction

In [ ]:
def capture_screenshot(output_path, episode, step):
    file_name = f"episode_{episode}_step_{step}.png"
    file_path = os.path.join(output_path, file_name)
    traci.gui.screenshot(traci.gui.DEFAULT_VIEW, file_path)


def agent_predict(env, model, episodes, save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    rewards_per_episode = []  # List to store rewards for each episode

    for episode in range(episodes):
        obs, info = env.reset()
        done = False
        step = 0
        total_reward = 0  # Variable to store total reward for the episode
        while step < 3600:
            action, _states = model.predict(obs)
            results = env.step(action)
            if len(results) == 5:
                obs, rewards, done, _, info = results
                if done:
                    print(f"DONE: {done}")
            else:
                raise ValueError(f"Expected 5 values from env.step(), got {len(results)}")
            total_reward += rewards  # Accumulate rewards
            # Capture and save the environment state every 10 steps
            # if step % 10 == 0:
            #     capture_screenshot(save_path, episode, step)
            step += 1
        rewards_per_episode.append(total_reward)  # Append total reward for this episode

    # Plot the rewards
    plt.plot(range(1, episodes + 1), rewards_per_episode, marker='o')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.title('Improvement in Total Reward over Episodes')
    plt.grid(True)
    plt.show()


if __name__ == "__main__":
    print("Connecting to SUMO server...")
    traci_connection_predict = traci.connect(port=65533)
    print("Connected to SUMO server")

    # Load the trained model
    loaded_model = DQN.load('Outputs/single-intersection/dqn20240604_165649.zip')
    agent_predict(env=env_dqn_agent, model=loaded_model, episodes=3, save_path='Outputs/single-intersection/dqn_images')
    traci.close()

# Naive Fixed-Time Solution

In [119]:
if __name__ == "__main__":
    print("Connecting to SUMO server...")
    traci_connection_predict = traci.connect(port=65533)
    print("Connected to SUMO server")

    # Load the trained model
    loaded_model = DQN.load('Navie_solution_model') # Load the naive solution model
    agent_predict(env=env_fixed_policy, model=loaded_model, episodes=3, save_path='Outputs/single-intersection/dqn_images')
    traci.close()

# Create a Video from the Captured Images

In [10]:
def create_video(output_path, title):
    images = []
    for file_name in sorted(os.listdir(output_path)):
        if file_name.endswith(".png"):
            file_path = os.path.join(output_path, file_name)
            images.append(imageio.imread(file_path))
    video_path = f'{title}.mp4'
    imageio.mimsave(video_path, images, fps=10)
    return Video(video_path)


create_video("outputs/single-intersection/dqn_images", "DQN Agent Solution")


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1430, 613) to (1440, 624) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


# Visualizing and Analyzing the Results

## Waiting time over episodes

In [ ]:

def plot_metrics(data, title, ax):
    ax.plot(data['step'], data['system_mean_waiting_time'], label='System Mean Waiting Time')
    ax.set_xlabel('Step Time')
    ax.set_ylabel('System Mean Waiting Time')
    ax.set_title(title)
    ax.legend()

def visualize_results():
    episodes = range(5, 85, 5)
    fig, axs = plt.subplots(len(episodes)//2, 2, figsize=(15, 20))
    axs = axs.flatten()

    for i, episode in enumerate(episodes):
        file_path = f"Outputs/single-intersection/dqn_20240612_164825.csv_conn1_ep{episode}.csv"
        title = f"DQN Agent Solution - Episode {episode}"
        
        try:
            data = pd.read_csv(file_path)
            if 'step' not in data.columns or 'system_mean_waiting_time' not in data.columns:
                raise KeyError("CSV file does not contain required columns: 'step' and 'system_mean_waiting_time'")
            
            plot_metrics(data, title, axs[i])
        
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except KeyError as e:
            print(e)

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    visualize_results()
    

## Calculate the Overall Average Waiting Time

### DQN Agent Solution and Naive Solution

In [ ]:
# change to def and send different path every time and range of episodes

mean_average_waiting_time = 0
list_of_average = []
episodes = list(range(5, 85))  # Define the correct range of episodes

# Load the new CSV file
for episode in episodes:
    file_path = f'Outputs/single-intersection/dqn_20240612_164825.csv_conn1_ep{episode}.csv'
    data = pd.read_csv(file_path)
    # Calculate the overall average waiting time
    average_waiting_time = data['system_mean_waiting_time'].mean()
    mean_average_waiting_time += average_waiting_time  
    list_of_average.append(average_waiting_time)

# Calculate the overall mean average waiting time
overall_mean_average_waiting_time = mean_average_waiting_time / len(episodes)
print(f'The overall average waiting time is: {overall_mean_average_waiting_time}\n')

# Plotting the average waiting time over episodes
plt.figure(figsize=(12, 8))
plt.plot(episodes, list_of_average, marker='o', linestyle='-', color='r', label='Avg Waiting Time')
plt.xlabel('Episode')
plt.ylabel('Average Waiting Time')
plt.title('Improvement in System Mean Waiting Time over Episodes')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.show()

# The END

In [ ]:
# write function to convert xml to csv
import xml.etree.ElementTree as ET
import pandas as pd

def xml_to_csv(xml_file_path, csv_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    data = {
        'id': [],
        'depart': [],
        'arrival': [],
        'waitingTime': []
    }

    for tripinfo in root.findall('tripinfo'):
        data['id'].append(tripinfo.get('id'))
        data['depart'].append(float(tripinfo.get('depart')))
        data['arrival'].append(float(tripinfo.get('arrival')))
        data['waitingTime'].append(float(tripinfo.get('waitingTime')))

    df = pd.DataFrame(data)
    df.to_csv(csv_file_path, index=False)
    print(f"CSV file saved as: {csv_file_path}")
    
file_to_convert = '/Users/md/Desktop/sumo-rl/nets/single-intersection/trip_info.xml'
csv_file_path = '/Users/md/Desktop/sumo-rl/nets/single-intersection/trip_info.csv'
xml_to_csv(file_to_convert, csv_file_path)